In [210]:
import tensorflow as tf
import cv2
import numpy as np
import os
import sys # for debugging 

In [211]:

# # This function preprocesses the image by reading in the image apply grayscale make all the sizes the same and 
# def preprocess_image(file_path, img_size):
#     img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE) # Grayscale will even the playing field if we start getting different types of images. If the images color is a factor we can take out grayscale
#     img = cv2.resize(img, img_size)
#     img = img.astype('float')/255.0 # Make the pixels become float and normalize to 0-1 for normalization
#     return img







# This function preprocesses the image by reading in the image apply grayscale make all the sizes the same and 
def preprocess_image(file_path, img_size):
    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE) # Grayscale will even the playing field if we start getting different types of images. If the images color is a factor we can take out grayscale
    
    # Thresholding to remove black background
    _, binary_image = cv2.threshold(img, 10, 255, cv2.THRESH_BINARY)
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary_image, connectivity=8)
    largest_component_label = np.argmax(stats[1:, cv2.CC_STAT_AREA]) + 1
    brain_mask = (labels == largest_component_label).astype(np.uint8) * 255
    x, y, w, h = cv2.boundingRect(brain_mask)
    img = img[y:y+h, x:x+w]
    
    img = cv2.resize(img, img_size)
    img = img.astype('float')/255.0 # Make the pixels become float and normalize to 0-1 for normalization
    return img


target_size =(224, 224)

# This function will pull from the directory and all subdirectory for the image and give it a label to the directory it is in
def load_images_from_directory(directory):
    images = []
    labels = []
    # Iterates through all subdirectories
    for subdir in os.listdir(directory):
        label = subdir #Make the subdirectory name be a label
        subdir_path = os.path.join(directory, subdir)

        # Checks if the object it is looking at is a directory and if it is go into the directory and get all the files and preprocess them
        if os.path.isdir(subdir_path):
            for image in os.listdir(subdir_path):
                file_path = os.path.join(subdir_path, image)

                image = preprocess_image(file_path, target_size)

                # Append to the arrays after preprocessing
                images.append(image)
                labels.append(label)

    return np.array(images), np.array(labels)

In [212]:


# Define the directory paths for the training and test datasets
train_dir = "./Alzheimer_s Dataset/train"
test_dir = "./Alzheimer_s Dataset/test"
# single_test_dir = "./Alzheimer_s Dataset/single_test"

# Load images and labels from the training directory
alz_images_train, alz_labels_train = load_images_from_directory(train_dir)

# Load images and labels from the test directory
alz_images_test, alz_labels_test = load_images_from_directory(test_dir)

# alz_single_images_test, alz_single_labels_test = load_images_from_directory(single_test_dir)

# Print information about the training dataset
print("Train")
print('Image shape:', alz_images_train.shape)
print('Labels shape:', alz_labels_train.shape)

# Print information about the test dataset
print("\nTest")
print('Image shape:', alz_images_test.shape)
print('Labels shape:', alz_labels_test.shape)


# print("\nSingle Test")
# print('Image shape:', alz_single_images_test.shape)
# print('Labels shape:', alz_single_labels_test.shape)


# np.set_printoptions(threshold=sys.maxsize) # for debugging

# print('Image train:', alz_single_images_test) # for debugging

# The output of the shape follows this
#  (X, X1, X2)
# X is the number of pictures in the array   
# X1 is the number of rows for a single picture (should be 224 since that is the scale)
# X2 is the number of columns in each picture  (should be 224 since that is the scale)
#  *Scale can be change to 207 since that is how the data is processed. 
# 
# When pull out the full array, you see alot of 0 at the start and end and that is because of the black around the brain

Train
Image shape: (5121, 224, 224)
Labels shape: (5121,)

Test
Image shape: (1279, 224, 224)
Labels shape: (1279,)


In [213]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

alz_labels_train_encoded = label_encoder.fit_transform(alz_labels_train)
alz_labels_test_encoded = label_encoder.fit_transform(alz_labels_test)

num_classes = len(label_encoder.classes_)

alz_labels_train_onehot = tf.keras.utils.to_categorical(alz_labels_train_encoded, num_classes)
alz_labels_test_onehot = tf.keras.utils.to_categorical(alz_labels_test_encoded, num_classes)

#np.set_printoptions(threshold=sys.maxsize) # for debugging
#print(alz_labels_train_onehot)

print("Training labels shape (one-hot encoded):", alz_labels_train_onehot.shape)
print("Testing labels shape (one-hot encoded):", alz_labels_test_onehot.shape)

# print('Image train:', alz_images_train) # for debugging


# 0 = MildDemented
# 1 = ModerateDemented
# 2 = NonDemented
# 3 = VeryMildDemented

Training labels shape (one-hot encoded): (5121, 4)
Testing labels shape (one-hot encoded): (1279, 4)


In [214]:
img_height = target_size[1]
img_width = target_size[0]
num_channels = 1

<h1>GCN preprocessing</h1>

In [8]:
from sklearn.decomposition import PCA

# Generator function to yield batches of preprocessed images
def image_generator(images, batch_size=32):
    num_images = len(images)
    num_batches = (num_images + batch_size - 1) // batch_size
    
    for i in range(num_batches):
        start_index = i * batch_size
        end_index = min((i + 1) * batch_size, num_images)
        
        # Load and preprocess images for the current batch
        batch_images = np.stack((images[start_index:end_index],) * 3, axis=-1)
        preprocessed_images = tf.keras.applications.mobilenet_v2.preprocess_input(batch_images)
        
        yield preprocessed_images

# Function to extract features from images using a pre-trained CNN and perform PCA
def extract_features(images, batch_size=32, n_components=64):
    feature_extractor = tf.keras.applications.MobileNetV2(include_top=False, weights='imagenet', input_shape=(img_height, img_width, 3))
    feature_extractor.trainable = False
    
    features = []
    for batch_images in image_generator(images, batch_size=batch_size):
        batch_features = feature_extractor.predict(batch_images)
        batch_features_flat = batch_features.reshape(batch_features.shape[0], -1)
        features.append(batch_features_flat)
    
    all_features = np.concatenate(features, axis=0)
    
    pca = PCA(n_components=n_components)
    reduced_features = pca.fit_transform(all_features)
    
    return reduced_features

# Extract features from training and test images
train_features = extract_features(alz_images_train)
test_features = extract_features(alz_images_test)



1/1 [==============================] - 1s 721ms/step


In [12]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Compute adjacency matrix based on feature similarity
def compute_feature_similarity(features):
    num_images = features.shape[0]
    similarities = np.zeros((num_images, num_images))
    for i in range(num_images):
        for j in range(num_images):
            # Compute cosine similarity between feature vectors
            similarities[i, j] = cosine_similarity(features[i].reshape(1, -1), features[j].reshape(1, -1))[0, 0]
    return similarities



In [17]:
# Compute feature similarities for training and test images
train_feature_similarity = compute_feature_similarity(train_features)

In [13]:
test_feature_similarity = compute_feature_similarity(test_features)

In [250]:
similarity_threshold = 0.8

# Construct adjacency matrix based on feature similarity
def construct_adjacency_matrix(feature_similarity, threshold):
    num_images = feature_similarity.shape[0]
    adjacency_matrix = np.zeros((num_images, num_images))
    for i in range(num_images):
        for j in range(num_images):
            # Set adjacency matrix value based on whether feature similarity is above threshold
            if feature_similarity[i, j] >= threshold:
                adjacency_matrix[i, j] = 1
                adjacency_matrix[j, i] = 1  # Symmetric adjacency matrix
    return adjacency_matrix


In [216]:
# Compute adjacency matrix for training and test images
train_adj_matrix = construct_adjacency_matrix(train_feature_similarity, similarity_threshold)
print("Shape of adjacency matrix for training images:", train_adj_matrix.shape)


Shape of adjacency matrix for training images: (5121, 5121)


In [217]:
test_adj_matrix = construct_adjacency_matrix(test_feature_similarity, similarity_threshold)
print("Shape of adjacency matrix for test images:", test_adj_matrix.shape)

Shape of adjacency matrix for test images: (1279, 1279)


In [218]:

print("Shape of train_features:", train_features.shape)
print("Shape of test_features:", test_features.shape)

print("\nShape of train_features similarity:", train_feature_similarity.shape)
print("Shape of test_features similarity:", test_feature_similarity.shape)

print("\nShape of train_adj_matrix:", train_adj_matrix.shape)
print("Shape of test_adj_matrix:", test_adj_matrix.shape)



Shape of train_features: (5121, 64)
Shape of test_features: (1279, 64)

Shape of train_features similarity: (5121, 5121)
Shape of test_features similarity: (1279, 1279)

Shape of train_adj_matrix: (5121, 5121)
Shape of test_adj_matrix: (1279, 1279)


In [223]:
# Images with grayscale index
alz_images_train_with_grayscale_index = alz_images_train[..., np.newaxis]  # Add channel dimension for grayscale images
alz_images_test_with_grayscale_index = alz_images_test[..., np.newaxis]

# Integrate graph-based features
train_features_with_graph = np.concatenate([train_features, train_adj_matrix], axis=1)
test_features_with_graph = np.concatenate([test_features, test_adj_matrix], axis=1)

print("Shape of training images with grayscale index:", alz_images_train_with_grayscale_index.shape)
print("Shape of testing images with grayscale index:", alz_images_test_with_grayscale_index.shape)
print("Shape of training features with graph:", train_features_with_graph.shape)
print("Shape of testing features with graph:", test_features_with_graph.shape)


Shape of training images with grayscale index: (5121, 224, 224, 1)
Shape of testing images with grayscale index: (1279, 224, 224, 1)
Shape of training features with graph: (5121, 5185)
Shape of testing features with graph: (1279, 1343)


<h1>CNN with graph based features </h1>

In [248]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Concatenate, Flatten, Dense

# Define input layers for image data and graph data
image_input = Input(shape=(img_height, img_width, 1), name='image_input')
graph_input = Input(shape=(train_features.shape[1],), name='graph_input')
adj_input = Input(shape=(train_adj_matrix.shape[1],), name='adj_input')
print(adj_input)

# Flatten the image data
flatten_image = Flatten()(image_input)

# Concatenate flattened image data with graph data
concatenated_input = Concatenate()([flatten_image, graph_input, adj_input])

# Define the dense layers
x = Dense(128, activation='relu')(concatenated_input)
x = Dense(64, activation='relu')(x)

# Output layer
output = Dense(4, activation='softmax', name='output')(x)

# Create the model
model2 = Model(inputs=[image_input, graph_input, adj_input], outputs=output)

# Compile the model
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


KerasTensor(type_spec=TensorSpec(shape=(None, 5121), dtype=tf.float32, name='adj_input'), name='adj_input', description="created by layer 'adj_input'")
Model: "model_31"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 image_input (InputLayer)       [(None, 224, 224, 1  0           []                               
                                )]                                                                
                                                                                                  
 flatten_5 (Flatten)            (None, 50176)        0           ['image_input[0][0]']            
                                                                                                  
 graph_input (InputLayer)       [(None, 64)]         0           []                               
                                      

In [249]:
# Define the number of epochs and batch size
epochs = 10
batch_size = 32

history = model.fit(
    {'image_input': alz_images_train_with_grayscale_index, 'graph_input': train_features, 'adj_input': train_adj_matrix},
    {'output': alz_labels_train_onehot},
    epochs=10,
    batch_size=32,
    validation_data=({'image_input': alz_images_test_with_grayscale_index, 'graph_input': test_features, 'adj_input': test_adj_matrix}, {'output': alz_labels_test_onehot})
)



Epoch 1/10


c:\Users\User\anaconda3\envs\TensorFlow\lib\site-packages\keras\engine\functional.py:637: UserWarning: Input dict contained keys ['adj_input'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


161/161 [==============================] - 16s 91ms/step - loss: 4.7009 - accuracy: 0.4716 - val_loss: 1.5796 - val_accuracy: 0.5387
Epoch 2/10
161/161 [==============================] - 14s 88ms/step - loss: 0.9648 - accuracy: 0.6249 - val_loss: 3.1012 - val_accuracy: 0.5020
Epoch 3/10
161/161 [==============================] - 14s 85ms/step - loss: 0.8954 - accuracy: 0.6616 - val_loss: 2.3167 - val_accuracy: 0.3612
Epoch 4/10
161/161 [==============================] - 14s 87ms/step - loss: 0.8329 - accuracy: 0.6848 - val_loss: 3.9748 - val_accuracy: 0.1650
Epoch 5/10
161/161 [==============================] - 14s 89ms/step - loss: 0.8538 - accuracy: 0.6633 - val_loss: 2.3657 - val_accuracy: 0.5012
Epoch 6/10
161/161 [==============================] - 15s 91ms/step - loss: 0.6276 - accuracy: 0.7405 - val_loss: 1.6394 - val_accuracy: 0.5536
Epoch 7/10
161/161 [==============================] - 14s 88ms/step - loss: 0.5937 - accuracy: 0.7479 - val_loss: 1.5371 - val_accuracy: 0.5590
Epo

<h1>FNN</h1>
still need to adjust a bit 

In [22]:
# Define the GCN model architecture
def create_gcn_model(input_dim, output_dim):
    inputs = tf.keras.Input(shape=(input_dim,))
    x = layers.Dense(64, activation='relu')(inputs)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(output_dim, activation='softmax')(x)  # Adjust activation based on your task
    model = tf.keras.Model(inputs=inputs, outputs=x)
    return model

# Define model parameters
input_dim = train_features.shape[1]  # Input dimension is the number of features after PCA
output_dim = num_classes  # Output dimension is the number of classes
learning_rate = 0.001
batch_size = 32
epochs = 10

# Create and compile the model
model = create_gcn_model(input_dim, output_dim)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_features, alz_labels_train_onehot, 
                    batch_size=batch_size, 
                    epochs=epochs, 
                    validation_data=(test_features, alz_labels_test_onehot))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_features, alz_labels_test_onehot)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')



Epoch 1/10
161/161 [==============================] - 1s 2ms/step - loss: 1.3894 - accuracy: 0.4583 - val_loss: 1.2713 - val_accuracy: 0.4496
Epoch 2/10
161/161 [==============================] - 0s 1ms/step - loss: 1.0049 - accuracy: 0.5558 - val_loss: 1.2862 - val_accuracy: 0.4464
Epoch 3/10
161/161 [==============================] - 0s 1ms/step - loss: 0.9077 - accuracy: 0.5948 - val_loss: 1.2831 - val_accuracy: 0.4371
Epoch 4/10
161/161 [==============================] - 0s 1ms/step - loss: 0.8381 - accuracy: 0.6335 - val_loss: 1.2883 - val_accuracy: 0.4425
Epoch 5/10
161/161 [==============================] - 0s 1ms/step - loss: 0.8034 - accuracy: 0.6407 - val_loss: 1.3201 - val_accuracy: 0.4347
Epoch 6/10
161/161 [==============================] - 0s 1ms/step - loss: 0.7784 - accuracy: 0.6493 - val_loss: 1.3408 - val_accuracy: 0.4308
Epoch 7/10
161/161 [==============================] - 0s 1ms/step - loss: 0.7555 - accuracy: 0.6563 - val_loss: 1.3790 - val_accuracy: 0.4167
Epoch 